In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv('creditcard.csv')
X=df.drop(columns=['Class'])
y=df['Class']


In [4]:
scaler=StandardScaler()
X[['Time','Amount']]=scaler.fit_transform(X[['Time', 'Amount']])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_normal=X_train[y_train==0]

In [10]:
input_dim=X_train_normal.shape[1]
input_layer=Input(shape=(input_dim,))
encoder=Dense(14,activation='relu')(input_layer)
encoder=Dense(7,activation='relu')(encoder)
latent=Dense(3,activation='relu')(encoder)

decoder=Dense(7,activation='relu')(latent)
decoder=Dense(14,activation='relu')(decoder)
Output_layer=Dense(input_dim,activation='linear')(decoder)

autoencoder=Model(inputs=input_layer,outputs=Output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001),loss='mse', metrics=['mae'])
autoencoder.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 30)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 14)                  │             434 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 7)                   │             105 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 3)                   │              24 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 7)                   │              28 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 14)                  │             112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 30)                  │             450 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,153 (4.50 KB)

 Trainable params: 1,153 (4.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
H=autoencoder.fit(
    X_train_normal,X_train_normal,
    epochs=50,
    batch_size=256,
    shuffle=True,
    validation_split=0.2,
    verbose=1
)

Epoch 1/50
622/622 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.9531 - mae: 0.6233 - val_loss: 0.6882 - val_mae: 0.5365
Epoch 2/50
622/622 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.6590 - mae: 0.5253 - val_loss: 0.6105 - val_mae: 0.5077
Epoch 3/50
622/622 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.6025 - mae: 0.5029 - val_loss: 0.5810 - val_mae: 0.4987
Epoch 4/50
622/622 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.5715 - mae: 0.4915 - val_loss: 0.5573 - val_mae: 0.4887
Epoch 5/50
622/622 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.5531 - mae: 0.4858 - val_loss: 0.5503 - val_mae: 0.4857
Epoch 6/50
622/622 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - loss: 0.5468 - mae: 0.4835 - val_loss: 0.5463 - val_mae: 0.4843
Epoch 7/50
622/622 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.5586 - mae: 0.4843 - val_loss: 0.5437 - val_mae: 0.4823
Epoch 8/50
622/622 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.5589 - mae: 0.4827 - val_loss: 0.5408 - val_mae: 0.4817
Epoch 9/50
622/622 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - 

In [15]:
X_train_normal_pred=autoencoder.predict(X_train_normal)
mse=np.mean(np.square(X_train_normal_pred - X_train_normal),axis=1)

threshold=np.percentile(mse,95)
print ("Threshold :" ,threshold)

6219/6219 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step
Threshold : 1.1367904742525805


In [16]:
X_test_pred=autoencoder.predict(X_test)
mse_test = np.mean(np.square(X_test_pred - X_test), axis=1)

y_pred = (mse_test > threshold).astype(int)


2671/2671 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [17]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[81070  4237]
 [   18   118]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     85307
           1       0.03      0.87      0.05       136

    accuracy                           0.95     85443
   macro avg       0.51      0.91      0.51     85443
weighted avg       1.00      0.95      0.97     85443

